In [1]:
import numpy as np
import pandas as pd
from untrade.client import Client
data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")

In [2]:
import pandas as pd
import numpy as np

def process_data(df):
    """
    Process the input dataframe to calculate indicators, generate trade signals,
    and apply stop-loss (SL) and take-profit (TP) logic based on the new strategy.

    Parameters:
    - df (DataFrame): Input dataframe with OHLC and volume data.

    Returns:
    - df (DataFrame): Processed dataframe with signals, trade types, and additional columns.
    """
    df = df.copy()
    
    # Calculate moving averages
    df['MA_13'] = df['close'].rolling(window=13).mean()
    df['MA_15'] = df['close'].rolling(window=15).mean()
    df['MA_200'] = df['close'].rolling(window=200).mean()
    
    # Calculate Bollinger Bands (20-period SMA with 2 standard deviations)
    df['BB_Mid'] = df['close'].rolling(window=20).mean()
    df['BB_Upper'] = df['BB_Mid'] + (2 * df['close'].rolling(window=20).std())
    df['BB_Lower'] = df['BB_Mid'] - (2 * df['close'].rolling(window=20).std())
    
    # Calculate average volume over the last 26 periods
    df['Avg_Volume'] = df['volume'].rolling(window=26).mean()
    
    # Initialize trade columns
    df['signals'] = 0  # Buy (1), Sell (-1), Square-off (0)
    df['trade'] = 0  # Track active trades (1 for long, -1 for short)
    df['SL'] = np.nan  # Stop-loss levels
    df['TP'] = np.nan  # Take-profit levels
    df['trade_type'] = 'square-off'  # Trade type ('long', 'short', 'square-off')
    
    # Loop through data to calculate signals and update trade states
    for i in range(200, len(df)):
        curr_close = df['close'].iloc[i]
        curr_low = df['low'].iloc[i]
        curr_high = df['high'].iloc[i]
        curr_vol = df['volume'].iloc[i]
        avg_vol = df['Avg_Volume'].iloc[i]
        prev_trade = df['trade'].iloc[i-1]
        
        # Check if volume condition is met
        vol_condition = curr_vol >= 1.3 * avg_vol
        
        # Long condition
        if df['MA_13'].iloc[i] > df['MA_15'].iloc[i] and curr_close > df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == -1:  # Square-off short trade
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open long trade
                delta = curr_close - curr_low
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'SL'] = curr_low - (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Upper'].iloc[i]
                df.loc[df.index[i], 'trade'] = 1
                df.loc[df.index[i], 'trade_type'] = 'long'
        
        # Short condition
        elif df['MA_13'].iloc[i] < df['MA_15'].iloc[i] and curr_close < df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == 1:  # Square-off long trade
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open short trade
                delta = curr_high - curr_close
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'SL'] = curr_high + (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Lower'].iloc[i]
                df.loc[df.index[i], 'trade'] = -1
                df.loc[df.index[i], 'trade_type'] = 'short'
        else:
            df.loc[df.index[i], 'trade'] = prev_trade  # Maintain trade state
    
    # Keep only required columns for backtesting
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 'SL', 'TP', 'MA_13', 'MA_15', 'MA_200', 'BB_Upper', 'BB_Lower']]
    
    return df



def strat(data):
    """
    Filter out consecutive duplicate signals to ensure no repetitive trades.

    Parameters:
    - data (DataFrame): Dataframe with trade signals.

    Returns:
    - data (DataFrame): Dataframe with filtered signals.
    """
    data = data.copy()
    signal = []
    prev = None
    for value in data["signals"]:
        if value == prev:
            signal.append(0)  # No action if same signal as previous
        else:
            signal.append(value)  # Keep the signal if different
        prev = value

    data["signals"] = signal
    

    # Retain only necessary columns
    data = data[['datetime', 'open', 'high', 'low',
                'close', 'volume', 'signals', 'trade_type']]
    
    
    data["leverage"] = 5
    # data["position"] = np.where(data["ATR_normalized"] < data["ATR_normalized"].quantile(0.5), 100, 50)
    return data


def perform_backtest(csv_file_path):
    """
    Perform backtesting using the Untrade SDK.

    Parameters:
    - csv_file_path (str): Path to the CSV file with processed data.

    Returns:
    - result (generator): Backtest results generator.
    """
    # Create Untrade client instance
    client = Client()

    # Perform backtest using Untrade
    result = client.backtest(
        jupyter_id="team64_zelta_hpps",  # User's Jupyter ID for Untrade
        file_path=csv_file_path,
        leverage=5,  # Set leverage for backtest
        # position=50,  # Set initial position sizeq
    )

    return result

# Load ETH data


data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 61,
      "Leverage Applied": 5.0,
      "Winning Trades": 24,
      "Losing Trades": 37,
      "No. of Long Trades": 30,
      "No. of Short Trades": 31,
      "Benchmark Return(%)": 1447.07322,
      "Benchmark Return(on $1000)": 14470.732205,
      "Win Rate": 39.344262,
      "Winning Streak": 6,
      "Losing Streak": 5,
      "Gross Profit": 17791.858056,
      "Net Profit": 17334.358056,
      "Average Profit": 284.169804,
      "Maximum Drawdown(%)": 35.034851,
      "Average Drawdown(%)": 6.265724,
      "Largest Win": 6364.174491,
      "Average Win": 1315.866931,
      "Largest Loss": -1635.451994,
      "Average Loss": -385.039143,
      "Maximum Holding Time": "94 days 0:0:0",
      "Average Holding Time": "21 days 5:14:45",
      "Maximum Adverse Excursion": 48.84106

In [3]:
import pandas as pd
import numpy as np

def process_data(df):
    """
    Process the input dataframe to calculate indicators, generate trade signals,
    and apply stop-loss (SL) and take-profit (TP) logic based on the new strategy.

    Parameters:
    - df (DataFrame): Input dataframe with OHLC and volume data.

    Returns:
    - df (DataFrame): Processed dataframe with signals, trade types, and additional columns.
    """
    df = df.copy()
    
    # Calculate moving averages
    df['MA_13'] = df['close'].rolling(window=13).mean()
    df['MA_15'] = df['close'].rolling(window=15).mean()
    df['MA_200'] = df['close'].rolling(window=200).mean()
    
    # Calculate Bollinger Bands (20-period SMA with 2 standard deviations)
    df['BB_Mid'] = df['close'].rolling(window=20).mean()
    df['BB_Upper'] = df['BB_Mid'] + (2 * df['close'].rolling(window=20).std())
    df['BB_Lower'] = df['BB_Mid'] - (2 * df['close'].rolling(window=20).std())
    
    # Calculate average volume over the last 26 periods
    df['Avg_Volume'] = df['volume'].rolling(window=26).mean()
    
    # Initialize trade columns
    df['signals'] = 0  # Buy (1), Sell (-1), Square-off (0)
    df['trade'] = 0  # Track active trades (1 for long, -1 for short)
    df['SL'] = np.nan  # Stop-loss levels
    df['TP'] = np.nan  # Take-profit levels
    df['trade_type'] = 'square-off'  # Trade type ('long', 'short', 'square-off')
    
    # Loop through data to calculate signals and update trade states
    for i in range(200, len(df)):
        curr_close = df['close'].iloc[i]
        curr_low = df['low'].iloc[i]
        curr_high = df['high'].iloc[i]
        curr_vol = df['volume'].iloc[i]
        avg_vol = df['Avg_Volume'].iloc[i]
        prev_trade = df['trade'].iloc[i-1]
        
        # Check if volume condition is met
        vol_condition = curr_vol >= 1.3 * avg_vol
        
        # Long condition
        if df['MA_13'].iloc[i] > df['MA_15'].iloc[i] and curr_close > df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == -1:  # Square-off short trade
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open long trade
                delta = curr_close - curr_low
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'SL'] = curr_low - (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Upper'].iloc[i]
                df.loc[df.index[i], 'trade'] = 1
                df.loc[df.index[i], 'trade_type'] = 'long'
        
        # Short condition
        elif df['MA_13'].iloc[i] < df['MA_15'].iloc[i] and curr_close < df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == 1:  # Square-off long trade
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open short trade
                delta = curr_high - curr_close
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'SL'] = curr_high + (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Lower'].iloc[i]
                df.loc[df.index[i], 'trade'] = -1
                df.loc[df.index[i], 'trade_type'] = 'short'
        else:
            df.loc[df.index[i], 'trade'] = prev_trade  # Maintain trade state
    
    # Keep only required columns for backtesting
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 
         'SL', 'TP', 'MA_13', 'MA_15', 'MA_200', 'BB_Upper', 'BB_Lower', 'BB_Mid']]
    
    return df



def strat(data):
    """
    Adjust position sizing based on trend strength, volatility, and recent win rate.
    
    Parameters:
    - data (DataFrame): Dataframe with trade signals.
    
    Returns:
    - data (DataFrame): Processed dataframe with position sizing logic.
    """
    data = data.copy()

    # Ensure signals are not repeated
    signal = []
    prev = None
    for value in data["signals"]:
        if value == prev:
            signal.append(0)  # No action if same signal as previous
        else:
            signal.append(value)  # Keep the signal if different
        prev = value
    data["signals"] = signal

    # **Trend Strength Factor** (Distance from MA200)
    data["distance_MA200"] = abs(data["close"] - data["MA_200"]) / data["MA_200"]

    # **Volatility Factor** (Bollinger Band Width)
    data["BB_Width"] = (data["BB_Upper"] - data["BB_Lower"]) / data["BB_Mid"]

    # **Win Rate Tracking** (Rolling % of Profitable Trades Over Last 10 Trades)
    data["win_rate"] = data["signals"].rolling(10).apply(lambda x: (x == 1).sum() / 10, raw=True)

    # **Position Sizing Logic**
    conditions = [
        (data["distance_MA200"] > 0.05) & (data["BB_Width"] < data["BB_Width"].quantile(0.7)) & (data["win_rate"] > 0.5),  # Strong trend, low volatility, good win rate
        (data["distance_MA200"] > 0.05) & (data["BB_Width"] > data["BB_Width"].quantile(0.7)),  # Strong trend, high volatility
        (data["distance_MA200"] < 0.05),  # Weak trend (near MA200)
        (data["win_rate"] < 0.3)  # Losing streak (reduce risk)
    ]
    position_values = [100, 70, 50, 50]  # Position % for each condition
    data["position"] = np.select(conditions, position_values, default=100)

    # Retain only necessary columns
    data = data[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 'position']].copy()

    data["leverage"] = 5  # Keep leverage constant

    return data


def perform_backtest(csv_file_path):
    """
    Perform backtesting using the Untrade SDK.

    Parameters:
    - csv_file_path (str): Path to the CSV file with processed data.

    Returns:
    - result (generator): Backtest results generator.
    """
    # Create Untrade client instance
    client = Client()

    # Perform backtest using Untrade
    result = client.backtest(
        jupyter_id="team64_zelta_hpps",  # User's Jupyter ID for Untrade
        file_path=csv_file_path,
        leverage=5,  # Set leverage for backtest
        # position=50,  # Set initial position sizeq
    )

    return result

# Load ETH data


data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 61,
      "Leverage Applied": 5.0,
      "Winning Trades": 24,
      "Losing Trades": 37,
      "No. of Long Trades": 30,
      "No. of Short Trades": 31,
      "Benchmark Return(%)": 1447.07322,
      "Benchmark Return(on $1000)": 14470.732205,
      "Win Rate": 39.344262,
      "Winning Streak": 6,
      "Losing Streak": 5,
      "Gross Profit": 10751.046441,
      "Net Profit": 10489.296441,
      "Average Profit": 171.955679,
      "Maximum Drawdown(%)": 17.517426,
      "Average Drawdown(%)": 5.733314,
      "Largest Win": 4454.922144,
      "Average Win": 781.119997,
      "Largest Loss": -1144.816396,
      "Average Loss": -223.177932,
      "Maximum Holding Time": "94 days 0:0:0",
      "Average Holding Time": "21 days 5:14:45",
      "Maximum Adverse Excursion": 48.841067

In [4]:
import pandas as pd
import numpy as np

def process_data(df):
    """
    Process the input dataframe to calculate indicators, generate trade signals,
    and apply stop-loss (SL) and take-profit (TP) logic based on the new strategy.

    Parameters:
    - df (DataFrame): Input dataframe with OHLC and volume data.

    Returns:
    - df (DataFrame): Processed dataframe with signals, trade types, and additional columns.
    """
    df = df.copy()
    
    # Calculate moving averages
    df['MA_13'] = df['close'].rolling(window=13).mean()
    df['MA_15'] = df['close'].rolling(window=15).mean()
    df['MA_200'] = df['close'].rolling(window=200).mean()
    
    # Calculate Bollinger Bands (20-period SMA with 2 standard deviations)
    df['BB_Mid'] = df['close'].rolling(window=20).mean()
    df['BB_Upper'] = df['BB_Mid'] + (2 * df['close'].rolling(window=20).std())
    df['BB_Lower'] = df['BB_Mid'] - (2 * df['close'].rolling(window=20).std())
    
    # Calculate average volume over the last 26 periods
    df['Avg_Volume'] = df['volume'].rolling(window=26).mean()
    
    # Initialize trade columns
    df['signals'] = 0  # Buy (1), Sell (-1), Square-off (0)
    df['trade'] = 0  # Track active trades (1 for long, -1 for short)
    df['SL'] = np.nan  # Stop-loss levels
    df['TP'] = np.nan  # Take-profit levels
    df['trade_type'] = 'square-off'  # Trade type ('long', 'short', 'square-off')
    
    # Loop through data to calculate signals and update trade states
    for i in range(200, len(df)):
        curr_close = df['close'].iloc[i]
        curr_low = df['low'].iloc[i]
        curr_high = df['high'].iloc[i]
        curr_vol = df['volume'].iloc[i]
        avg_vol = df['Avg_Volume'].iloc[i]
        prev_trade = df['trade'].iloc[i-1]
        
        # Check if volume condition is met
        vol_condition = curr_vol >= 1.3 * avg_vol
        
        # Long condition
        if df['MA_13'].iloc[i] > df['MA_15'].iloc[i] and curr_close > df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == -1:  # Square-off short trade
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open long trade
                delta = curr_close - curr_low
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'SL'] = curr_low - (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Upper'].iloc[i]
                df.loc[df.index[i], 'trade'] = 1
                df.loc[df.index[i], 'trade_type'] = 'long'
        
        # Short condition
        elif df['MA_13'].iloc[i] < df['MA_15'].iloc[i] and curr_close < df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == 1:  # Square-off long trade
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open short trade
                delta = curr_high - curr_close
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'SL'] = curr_high + (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Lower'].iloc[i]
                df.loc[df.index[i], 'trade'] = -1
                df.loc[df.index[i], 'trade_type'] = 'short'
        else:
            df.loc[df.index[i], 'trade'] = prev_trade  # Maintain trade state
    
    # Keep only required columns for backtesting
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 
         'SL', 'TP', 'MA_13', 'MA_15', 'MA_200', 'BB_Upper', 'BB_Lower', 'BB_Mid']]
    
    
    return df



def strat(data):
    """
    Adjust leverage dynamically based on market conditions.
    
    Parameters:
    - data (DataFrame): Dataframe with trade signals.
    
    Returns:
    - data (DataFrame): Processed dataframe with leverage adjustments.
    """
    data = data.copy()

    # Ensure signals are not repeated
    signal = []
    prev = None
    for value in data["signals"]:
        if value == prev:
            signal.append(0)  # No action if same signal as previous
        else:
            signal.append(value)  # Keep the signal if different
        prev = value
    data["signals"] = signal

    # **Trend Strength Factor** (Distance from MA200)
    data["distance_MA200"] = abs(data["close"] - data["MA_200"]) / data["MA_200"]

    # **Volatility Factor** (Bollinger Band Width)
    data["BB_Width"] = (data["BB_Upper"] - data["BB_Lower"]) / data["BB_Mid"]

    # **Win Rate Tracking** (Rolling % of Profitable Trades Over Last 10 Trades)
    data["win_rate"] = data["signals"].rolling(10).apply(lambda x: np.mean(x[x > 0]) if np.any(x > 0) else 0, raw=True)

    # **Leverage Adjustment Rules**
    conditions = [
        (data["distance_MA200"] > 0.05) & (data["BB_Width"] < data["BB_Width"].quantile(0.7)) & (data["win_rate"] > 0.5),  # Strong trend, low volatility, good win rate
        (data["distance_MA200"] > 0.05) & (data["BB_Width"] > data["BB_Width"].quantile(0.7)),  # Strong trend, high volatility
        (data["distance_MA200"] < 0.05),  # Weak trend (near MA200)
        (data["win_rate"] < 0.3)  # Losing streak (reduce risk)
    ]
    leverage_values = [10, 5, 3, 2]  # Leverage values for each condition
    data["leverage"] = np.select(conditions, leverage_values, default=5)

    # Retain only necessary columns
    data = data[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 'leverage']].copy()

    return data


def perform_backtest(csv_file_path):
    """
    Perform backtesting using the Untrade SDK.

    Parameters:
    - csv_file_path (str): Path to the CSV file with processed data.

    Returns:
    - result (generator): Backtest results generator.
    """
    # Create Untrade client instance
    client = Client()

    # Perform backtest using Untrade
    result = client.backtest(
        jupyter_id="team64_zelta_hpps",  # User's Jupyter ID for Untrade
        file_path=csv_file_path,
        leverage=5,  # Set leverage for backtest
        # position=50,  # Set initial position sizeq
    )

    return result

# Load ETH data


data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 61,
      "Leverage Applied": 5.0,
      "Winning Trades": 24,
      "Losing Trades": 37,
      "No. of Long Trades": 30,
      "No. of Short Trades": 31,
      "Benchmark Return(%)": 1447.07322,
      "Benchmark Return(on $1000)": 14470.732205,
      "Win Rate": 39.344262,
      "Winning Streak": 6,
      "Losing Streak": 5,
      "Gross Profit": 21129.926917,
      "Net Profit": 20732.426917,
      "Average Profit": 339.875851,
      "Maximum Drawdown(%)": 21.020911,
      "Average Drawdown(%)": 5.963119,
      "Largest Win": 6364.174491,
      "Average Win": 1379.112916,
      "Largest Loss": -1635.451994,
      "Average Loss": -334.223867,
      "Maximum Holding Time": "94 days 0:0:0",
      "Average Holding Time": "21 days 5:14:45",
      "Maximum Adverse Excursion": 48.84106

In [5]:
import pandas as pd
import numpy as np

def process_data(df):
    """
    Process the input dataframe to calculate indicators, generate trade signals,
    and apply stop-loss (SL) and take-profit (TP) logic based on the new strategy.

    Parameters:
    - df (DataFrame): Input dataframe with OHLC and volume data.

    Returns:
    - df (DataFrame): Processed dataframe with signals, trade types, and additional columns.
    """
    df = df.copy()
    
    # Calculate moving averages
    df['MA_13'] = df['close'].rolling(window=13).mean()
    df['MA_15'] = df['close'].rolling(window=15).mean()
    df['MA_200'] = df['close'].rolling(window=200).mean()
    
    # Calculate Bollinger Bands (20-period SMA with 2 standard deviations)
    df['BB_Mid'] = df['close'].rolling(window=20).mean()
    df['BB_Upper'] = df['BB_Mid'] + (2 * df['close'].rolling(window=20).std())
    df['BB_Lower'] = df['BB_Mid'] - (2 * df['close'].rolling(window=20).std())
    
    # Calculate average volume over the last 26 periods
    df['Avg_Volume'] = df['volume'].rolling(window=26).mean()
    
    # Initialize trade columns
    df['signals'] = 0  # Buy (1), Sell (-1), Square-off (0)
    df['trade'] = 0  # Track active trades (1 for long, -1 for short)
    df['SL'] = np.nan  # Stop-loss levels
    df['TP'] = np.nan  # Take-profit levels
    df['trade_type'] = 'square-off'  # Trade type ('long', 'short', 'square-off')
    
    # Loop through data to calculate signals and update trade states
    for i in range(200, len(df)):
        curr_close = df['close'].iloc[i]
        curr_low = df['low'].iloc[i]
        curr_high = df['high'].iloc[i]
        curr_vol = df['volume'].iloc[i]
        avg_vol = df['Avg_Volume'].iloc[i]
        prev_trade = df['trade'].iloc[i-1]
        
        # Check if volume condition is met
        vol_condition = curr_vol >= 1.3 * avg_vol
        
        # Long condition
        if df['MA_13'].iloc[i] > df['MA_15'].iloc[i] and curr_close > df['MA_200'].iloc[i] and vol_condition:
            delta = curr_close - curr_low
            df.loc[df.index[i], 'SL'] = curr_low - (0.1 * delta)
            df.loc[df.index[i], 'TP'] = df['BB_Upper'].iloc[i]
            if prev_trade == -1:  # Square-off short trade
                df.loc[df.index[i], 'signals'] = -2
                df.loc[df.index[i], 'trade'] = 1
                df.loc[df.index[i], 'trade_type'] = 'long'
            else:  # Open long trade
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'trade'] = 1
                df.loc[df.index[i], 'trade_type'] = 'long'
        
        # Short condition
        elif df['MA_13'].iloc[i] < df['MA_15'].iloc[i] and curr_close < df['MA_200'].iloc[i] and vol_condition:
            delta = curr_high - curr_close
            df.loc[df.index[i], 'SL'] = curr_high + (0.1 * delta)
            df.loc[df.index[i], 'TP'] = df['BB_Lower'].iloc[i]
            if prev_trade == 1:  # Square-off long trade
                df.loc[df.index[i], 'signals'] = 2
                df.loc[df.index[i], 'trade'] = -1
                df.loc[df.index[i], 'trade_type'] = 'short'
            else:  # Open short trade
                
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'trade'] = -1
                df.loc[df.index[i], 'trade_type'] = 'short'
        else:
            df.loc[df.index[i], 'trade'] = prev_trade  # Maintain trade state
    
    # Keep only required columns for backtesting
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 
         'SL', 'TP', 'MA_13', 'MA_15', 'MA_200', 'BB_Upper', 'BB_Lower', 'BB_Mid']]
    
    
    return df

def strat(data):
    """
    Adjust leverage dynamically and improve entry price logic.

    Parameters:
    - data (DataFrame): Dataframe with trade signals.

    Returns:
    - data (DataFrame): Processed dataframe with leverage adjustments and optimized entry prices.
    """
    data = data.copy()

    # Ensure signals are not repeated
    signal = []
    prev = None
    for value in data["signals"]:
        if value == prev:
            signal.append(0)  # No action if same signal as previous
        else:
            signal.append(value)  # Keep the signal if different
        prev = value
    data["signals"] = signal

    # **Trend Strength Factor** (Distance from MA200)
    data["distance_MA200"] = abs(data["close"] - data["MA_200"]) / data["MA_200"]

    # **Volatility Factor** (Bollinger Band Width)
    data["BB_Width"] = (data["BB_Upper"] - data["BB_Lower"]) / data["BB_Mid"]

    # **ATR for Dynamic Entry Price**
    data["ATR"] = data["high"].rolling(14).max() - data["low"].rolling(14).min()

    # **Win Rate Tracking** (Rolling % of Profitable Trades Over Last 10 Trades)
    data["win_rate"] = data["signals"].rolling(10).apply(lambda x: np.mean(x[x > 0]) if np.any(x > 0) else 0, raw=True)

    # **Leverage Adjustment Rules**
    conditions = [
        (data["distance_MA200"] > 0.05) & (data["BB_Width"] < data["BB_Width"].quantile(0.7)) & (data["win_rate"] > 0.5),  # Strong trend, low volatility, good win rate
        (data["distance_MA200"] > 0.05) & (data["BB_Width"] > data["BB_Width"].quantile(0.7)),  # Strong trend, high volatility
        (data["distance_MA200"] < 0.05),  # Weak trend (near MA200)
        (data["win_rate"] < 0.3)  # Losing streak (reduce risk)
    ]
    leverage_values = [10, 7, 3, 2]  # Leverage values for each condition
    data["leverage"] = np.select(conditions, leverage_values, default=5)

    # **Optimized Entry Price Logic**
    entry_prices = []
    for i in range(len(data)):
        if data["signals"].iloc[i] == 1:  # Long trade
            entry_price = min(data["BB_Lower"].iloc[i] + 0.5 * data["ATR"].iloc[i], data["close"].iloc[i])
        elif data["signals"].iloc[i] == -1:  # Short trade
            entry_price = max(data["BB_Upper"].iloc[i] - 0.5 * data["ATR"].iloc[i], data["close"].iloc[i])
        else:
            entry_price = data["close"].iloc[i]  # No trade, keep normal price
        entry_prices.append(entry_price)

    # data["optimized_entry"] = entry_prices

    # Retain only necessary columns
    data = data[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 'leverage']].copy()

    return data


def perform_backtest(csv_file_path):
    """
    Perform backtesting using the Untrade SDK.

    Parameters:
    - csv_file_path (str): Path to the CSV file with processed data.

    Returns:
    - result (generator): Backtest results generator.
    """
    # Create Untrade client instance
    client = Client()

    # Perform backtest using Untrade
    result = client.backtest(
        jupyter_id="team64_zelta_hpps",  # User's Jupyter ID for Untrade
        file_path=csv_file_path,
        # leverage=5,  # Set leverage for backtest
        # position=50,  # Set initial position sizeq
    )

    return result

# Load ETH data


data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 131,
      "Leverage Applied": 1.0,
      "Winning Trades": 44,
      "Losing Trades": 87,
      "No. of Long Trades": 65,
      "No. of Short Trades": 66,
      "Benchmark Return(%)": 1447.07322,
      "Benchmark Return(on $1000)": 14470.732205,
      "Win Rate": 33.587786,
      "Winning Streak": 3,
      "Losing Streak": 8,
      "Gross Profit": 25572.494872,
      "Net Profit": 24738.494872,
      "Average Profit": 188.843472,
      "Maximum Drawdown(%)": 21.233502,
      "Average Drawdown(%)": 6.089037,
      "Largest Win": 10382.473396,
      "Average Win": 984.23271,
      "Largest Loss": -2096.54336,
      "Average Loss": -213.422349,
      "Maximum Holding Time": "93 days 3:59:59",
      "Average Holding Time": "9 days 20:7:19",
      "Maximum Adverse Excursion": 48.84106

In [6]:
import pandas as pd
import numpy as np

def process_data(df):
    """
    Process the input dataframe to calculate indicators, generate trade signals,
    and apply stop-loss (SL) and take-profit (TP) logic based on the new strategy.

    Parameters:
    - df (DataFrame): Input dataframe with OHLC and volume data.

    Returns:
    - df (DataFrame): Processed dataframe with signals, trade types, and additional columns.
    """
    df = df.copy()
    
    # Calculate moving averages
    df['MA_13'] = df['close'].rolling(window=13).mean()
    df['MA_15'] = df['close'].rolling(window=15).mean()
    df['MA_200'] = df['close'].rolling(window=200).mean()
    
    # Calculate Bollinger Bands (20-period SMA with 2 standard deviations)
    df['BB_Mid'] = df['close'].rolling(window=20).mean()
    df['BB_Upper'] = df['BB_Mid'] + (2 * df['close'].rolling(window=20).std())
    df['BB_Lower'] = df['BB_Mid'] - (2 * df['close'].rolling(window=20).std())
    
    # Calculate average volume over the last 26 periods
    df['Avg_Volume'] = df['volume'].rolling(window=26).mean()
    
    # Initialize trade columns
    df['signals'] = 0  # Buy (1), Sell (-1), Square-off (0)
    df['trade'] = 0  # Track active trades (1 for long, -1 for short)
    df['SL'] = np.nan  # Stop-loss levels
    df['TP'] = np.nan  # Take-profit levels
    df['trade_type'] = 'square-off'  # Trade type ('long', 'short', 'square-off')
    
    # Loop through data to calculate signals and update trade states
    for i in range(200, len(df)):
        curr_close = df['close'].iloc[i]
        curr_low = df['low'].iloc[i]
        curr_high = df['high'].iloc[i]
        curr_vol = df['volume'].iloc[i]
        avg_vol = df['Avg_Volume'].iloc[i]
        prev_trade = df['trade'].iloc[i-1]
        
        # Check if volume condition is met
        vol_condition = curr_vol >= 1.3 * avg_vol
        
        # Long condition
        if df['MA_13'].iloc[i] > df['MA_15'].iloc[i] and curr_close > df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == -1:  # Square-off short trade
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open long trade
                delta = curr_close - curr_low
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'SL'] = curr_low - (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Upper'].iloc[i]
                df.loc[df.index[i], 'trade'] = 1
                df.loc[df.index[i], 'trade_type'] = 'long'
        
        # Short condition
        elif df['MA_13'].iloc[i] < df['MA_15'].iloc[i] and curr_close < df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == 1:  # Square-off long trade
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open short trade
                delta = curr_high - curr_close
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'SL'] = curr_high + (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Lower'].iloc[i]
                df.loc[df.index[i], 'trade'] = -1
                df.loc[df.index[i], 'trade_type'] = 'short'
        else:
            df.loc[df.index[i], 'trade'] = prev_trade  # Maintain trade state
    
    # Keep only required columns for backtesting
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 
         'SL', 'TP', 'MA_13', 'MA_15', 'MA_200', 'BB_Upper', 'BB_Lower', 'BB_Mid']]
    
    
    return df

def strat(data):
    """
    Adjust leverage dynamically and improve entry price logic.

    Parameters:
    - data (DataFrame): Dataframe with trade signals.

    Returns:
    - data (DataFrame): Processed dataframe with leverage adjustments and optimized entry prices.
    """
    data = data.copy()

    # Ensure signals are not repeated
    signal = []
    prev = None
    for value in data["signals"]:
        if value == prev:
            signal.append(0)  # No action if same signal as previous
        else:
            signal.append(value)  # Keep the signal if different
        prev = value
    data["signals"] = signal

    # **Trend Strength Factor** (Distance from MA200)
    data["distance_MA200"] = abs(data["close"] - data["MA_200"]) / data["MA_200"]

    # **Volatility Factor** (Bollinger Band Width)
    data["BB_Width"] = (data["BB_Upper"] - data["BB_Lower"]) / data["BB_Mid"]

    # **ATR for Dynamic Entry Price**
    data["ATR"] = data["high"].rolling(14).max() - data["low"].rolling(14).min()

    # **Win Rate Tracking** (Rolling % of Profitable Trades Over Last 10 Trades)
    data["win_rate"] = data["signals"].rolling(10).apply(lambda x: np.mean(x[x > 0]) if np.any(x > 0) else 0, raw=True)

    # **Leverage Adjustment Rules**
    conditions = [
        (data["distance_MA200"] > 0.05) & (data["BB_Width"] < data["BB_Width"].quantile(0.7)) & (data["win_rate"] > 0.5),  # Strong trend, low volatility, good win rate
        (data["distance_MA200"] > 0.05) & (data["BB_Width"] > data["BB_Width"].quantile(0.7)),  # Strong trend, high volatility
        (data["distance_MA200"] < 0.05),  # Weak trend (near MA200)
        (data["win_rate"] < 0.3)  # Losing streak (reduce risk)
    ]
    leverage_values = [10, 7, 3, 2]  # Leverage values for each condition
    data["leverage"] = np.select(conditions, leverage_values, default=5)

    # **Optimized Entry Price Logic**
    entry_prices = []
    for i in range(len(data)):
        if data["signals"].iloc[i] == 1:  # Long trade
            entry_price = min(data["BB_Lower"].iloc[i] + 0.5 * data["ATR"].iloc[i], data["close"].iloc[i])
        elif data["signals"].iloc[i] == -1:  # Short trade
            entry_price = max(data["BB_Upper"].iloc[i] - 0.5 * data["ATR"].iloc[i], data["close"].iloc[i])
        else:
            entry_price = data["close"].iloc[i]  # No trade, keep normal price
        entry_prices.append(entry_price)

    data["optimized_entry"] = entry_prices

    # Retain only necessary columns
    data = data[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 'leverage', 'optimized_entry']].copy()

    return data


def perform_backtest(csv_file_path):
    """
    Perform backtesting using the Untrade SDK.

    Parameters:
    - csv_file_path (str): Path to the CSV file with processed data.

    Returns:
    - result (generator): Backtest results generator.
    """
    # Create Untrade client instance
    client = Client()

    # Perform backtest using Untrade
    result = client.backtest(
        jupyter_id="team64_zelta_hpps",  # User's Jupyter ID for Untrade
        file_path=csv_file_path,
        # leverage=5,  # Set leverage for backtest
        # position=50,  # Set initial position sizeq
    )

    return result

# Load ETH data


data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/ETH/ETHUSDT_4h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)


data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-12-01 00:00:00",
      "Total Trades": 61,
      "Leverage Applied": 1.0,
      "Winning Trades": 24,
      "Losing Trades": 37,
      "No. of Long Trades": 30,
      "No. of Short Trades": 31,
      "Benchmark Return(%)": 1447.07322,
      "Benchmark Return(on $1000)": 14470.732205,
      "Win Rate": 39.344262,
      "Winning Streak": 6,
      "Losing Streak": 5,
      "Gross Profit": 24840.161743,
      "Net Profit": 24376.661743,
      "Average Profit": 399.617406,
      "Maximum Drawdown(%)": 21.020911,
      "Average Drawdown(%)": 6.012934,
      "Largest Win": 8909.844288,
      "Average Win": 1625.485979,
      "Largest Loss": -2289.632791,
      "Average Loss": -395.540588,
      "Maximum Holding Time": "94 days 0:0:0",
      "Average Holding Time": "21 days 5:14:45",
      "Maximum Adverse Excursion": 48.84106

In [7]:
import pandas as pd
import numpy as np

def process_data(df):
    """
    Process the input dataframe to calculate indicators, generate trade signals,
    and apply stop-loss (SL) and take-profit (TP) logic based on the new strategy.

    Parameters:
    - df (DataFrame): Input dataframe with OHLC and volume data.

    Returns:
    - df (DataFrame): Processed dataframe with signals, trade types, and additional columns.
    """
    df = df.copy()
    
    # Calculate moving averages
    df['MA_13'] = df['close'].rolling(window=13).mean()
    df['MA_15'] = df['close'].rolling(window=15).mean()
    df['MA_200'] = df['close'].rolling(window=200).mean()
    
    # Calculate Bollinger Bands (20-period SMA with 2 standard deviations)
    df['BB_Mid'] = df['close'].rolling(window=20).mean()
    df['BB_Upper'] = df['BB_Mid'] + (2 * df['close'].rolling(window=20).std())
    df['BB_Lower'] = df['BB_Mid'] - (2 * df['close'].rolling(window=20).std())
    
    # Calculate average volume over the last 26 periods
    df['Avg_Volume'] = df['volume'].rolling(window=26).mean()
    
    # Initialize trade columns
    df['signals'] = 0  # Buy (1), Sell (-1), Square-off (0)
    df['trade'] = 0  # Track active trades (1 for long, -1 for short)
    df['SL'] = np.nan  # Stop-loss levels
    df['TP'] = np.nan  # Take-profit levels
    df['trade_type'] = 'square-off'  # Trade type ('long', 'short', 'square-off')
    
    # Loop through data to calculate signals and update trade states
    for i in range(200, len(df)):
        curr_close = df['close'].iloc[i]
        curr_low = df['low'].iloc[i]
        curr_high = df['high'].iloc[i]
        curr_vol = df['volume'].iloc[i]
        avg_vol = df['Avg_Volume'].iloc[i]
        prev_trade = df['trade'].iloc[i-1]
        
        # Check if volume condition is met
        vol_condition = curr_vol >= 1.3 * avg_vol
        
        # Long condition
        if df['MA_13'].iloc[i] > df['MA_15'].iloc[i] and curr_close > df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == -1:  # Square-off short trade
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open long trade
                delta = curr_close - curr_low
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'SL'] = curr_low - (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Upper'].iloc[i]
                df.loc[df.index[i], 'trade'] = 1
                df.loc[df.index[i], 'trade_type'] = 'long'
        
        # Short condition
        elif df['MA_13'].iloc[i] < df['MA_15'].iloc[i] and curr_close < df['MA_200'].iloc[i] and vol_condition:
            if prev_trade == 1:  # Square-off long trade
                df.loc[df.index[i], 'signals'] = 1
                df.loc[df.index[i], 'trade'] = 0
                df.loc[df.index[i], 'trade_type'] = 'square-off'
            else:  # Open short trade
                delta = curr_high - curr_close
                df.loc[df.index[i], 'signals'] = -1
                df.loc[df.index[i], 'SL'] = curr_high + (0.1 * delta)
                df.loc[df.index[i], 'TP'] = df['BB_Lower'].iloc[i]
                df.loc[df.index[i], 'trade'] = -1
                df.loc[df.index[i], 'trade_type'] = 'short'
        else:
            df.loc[df.index[i], 'trade'] = prev_trade  # Maintain trade state
    
    # Keep only required columns for backtesting
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 
         'SL', 'TP', 'MA_13', 'MA_15', 'MA_200', 'BB_Upper', 'BB_Lower', 'BB_Mid']]
    
    
    return df

def strat(data):
    """
    Adjust leverage dynamically and improve entry price logic.

    Parameters:
    - data (DataFrame): Dataframe with trade signals.

    Returns:
    - data (DataFrame): Processed dataframe with leverage adjustments and optimized entry prices.
    """
    data = data.copy()

    # Ensure signals are not repeated
    signal = []
    prev = None
    for value in data["signals"]:
        if value == prev:
            signal.append(0)  # No action if same signal as previous
        else:
            signal.append(value)  # Keep the signal if different
        prev = value
    data["signals"] = signal

    # **Trend Strength Factor** (Distance from MA200)
    data["distance_MA200"] = abs(data["close"] - data["MA_200"]) / data["MA_200"]

    # **Volatility Factor** (Bollinger Band Width)
    data["BB_Width"] = (data["BB_Upper"] - data["BB_Lower"]) / data["BB_Mid"]

    # **ATR for Dynamic Entry Price**
    data["ATR"] = data["high"].rolling(14).max() - data["low"].rolling(14).min()

    # **Win Rate Tracking** (Rolling % of Profitable Trades Over Last 10 Trades)
    data["win_rate"] = data["signals"].rolling(10).apply(lambda x: np.mean(x[x > 0]) if np.any(x > 0) else 0, raw=True)

    # **Leverage Adjustment Rules**
    conditions = [
        (data["distance_MA200"] > 0.05) & (data["BB_Width"] < data["BB_Width"].quantile(0.7)) & (data["win_rate"] > 0.5),  # Strong trend, low volatility, good win rate
        (data["distance_MA200"] > 0.05) & (data["BB_Width"] > data["BB_Width"].quantile(0.7)),  # Strong trend, high volatility
        (data["distance_MA200"] < 0.05),  # Weak trend (near MA200)
        (data["win_rate"] < 0.3)  # Losing streak (reduce risk)
    ]
    leverage_values = [10, 7, 3, 2]  # Leverage values for each condition
    data["leverage"] = np.select(conditions, leverage_values, default=5)

    # **Optimized Entry Price Logic**
    entry_prices = []
    for i in range(len(data)):
        if data["signals"].iloc[i] == 1:  # Long trade
            entry_price = min(data["BB_Lower"].iloc[i] + 0.5 * data["ATR"].iloc[i], data["close"].iloc[i])
        elif data["signals"].iloc[i] == -1:  # Short trade
            entry_price = max(data["BB_Upper"].iloc[i] - 0.5 * data["ATR"].iloc[i], data["close"].iloc[i])
        else:
            entry_price = data["close"].iloc[i]  # No trade, keep normal price
        entry_prices.append(entry_price)

    data["optimized_entry"] = entry_prices

    # Retain only necessary columns
    data = data[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals', 'trade_type', 'leverage', 'optimized_entry']].copy()

    return data


def perform_backtest(csv_file_path):
    """
    Perform backtesting using the Untrade SDK.

    Parameters:
    - csv_file_path (str): Path to the CSV file with processed data.

    Returns:
    - result (generator): Backtest results generator.
    """
    # Create Untrade client instance
    client = Client()

    # Perform backtest using Untrade
    result = client.backtest(
        jupyter_id="team64_zelta_hpps",  # User's Jupyter ID for Untrade
        file_path=csv_file_path,
        # leverage=5,  # Set leverage for backtest
        # position=50,  # Set initial position sizeq
    )

    return result

# Load ETH data


data = pd.read_csv("/Users/tejasmacipad/Desktop/Final_inter_IIT_submission/BTC/BTC_2019_2023_4h.csv")


# Process data
res = process_data(data)

# Apply filtering strategy to remove duplicate signals
res = strat(res)

# Save processed data to CSV for backtesting
res.to_csv("processed_data.csv", index=False)

# Perform backtesting
csv_file_path = "processed_data.csv"
backtest_result = perform_backtest(csv_file_path)

# Print backtest results
for value in backtest_result:
    print(value)

data: {
  "jupyter_id": "team64-5fzelta-5fhpps",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-09-08 16:00:00",
      "Total Trades": 79,
      "Leverage Applied": 1.0,
      "Winning Trades": 22,
      "Losing Trades": 57,
      "No. of Long Trades": 39,
      "No. of Short Trades": 40,
      "Benchmark Return(%)": 309.761202,
      "Benchmark Return(on $1000)": 3097.61202,
      "Win Rate": 27.848101,
      "Winning Streak": 3,
      "Losing Streak": 7,
      "Gross Profit": 10001.872772,
      "Net Profit": 9466.372772,
      "Average Profit": 119.827503,
      "Maximum Drawdown(%)": 94.518442,
      "Average Drawdown(%)": 16.137663,
      "Largest Win": 5740.352813,
      "Average Win": 1071.052329,
      "Largest Loss": -928.450589,
      "Average Loss": -247.311903,
      "Maximum Holding Time": "107 days 0:0:0",
      "Average Holding Time": "17 days 1:49:22",
      "Maximum Adverse Excursion": 42.274929,